In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 

/Users/ayan/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
! pwd

/Users/ayan/Programs/U_Geneva_Francesco/Folder_12_Pve_Def_Pass


In [3]:
def is_positive_definite(C, tol=1e-14):
    """
    Returns True if all eigenvalues of (the symmetric part of) C 
    are greater than 'tol'.
    """
    # Symmetrize to avoid issues with small numerical asymmetries
    C_sym = 0.5 * (C + C.T)
    eigenvals = np.linalg.eigvalsh(C_sym)
    return np.all(eigenvals > tol)


def read_covariance_matrix(file_path):
    with open(file_path, 'r') as file:
        # Read the size of the matrix
        n = int(file.readline().strip())
        
        # Initialize a placeholder for the matrix values
        matrix_values = []
        
        # Read the rest of the file line by line
        for line in file:
            # Split by '#' to separate value and comment
            value = line.split('#')[0].strip()
            if value:  # If there's a value present, add it
                matrix_values.append(float(value))
        
        # Ensure we have the correct number of values
        assert len(matrix_values) == n * n, "Matrix size mismatch!"
        
        # Reconstruct the matrix as a 2D list
        matrix = [matrix_values[i * n:(i + 1) * n] for i in range(n)]
        
        return matrix

    
    
### Eigen value clipping

def symmetrize(C):
    return 0.5 * (C + C.T)

def clip_eigenvalues(C, min_eigval=1e-12):
    """
    Force any eigenvalues of C below 'min_eigval' to be set to 'min_eigval',
    ensuring the matrix is (at least) positive definite.
    """
    # Symmetrize first to avoid small asymmetries
    C_sym = symmetrize(C)
    eigvals, eigvecs = np.linalg.eigh(C_sym)
    
    # Clip eigenvalues
    clipped_eigvals = np.maximum(eigvals, min_eigval)
    
    # Reconstruct
    C_clipped = eigvecs @ np.diag(clipped_eigvals) @ eigvecs.T
    return C_clipped


# Frobenius change test
# < 1% change on global difference is ok

def relative_frobenius_change(C_orig, C_clipped):
    diff = C_clipped - C_orig
    return np.linalg.norm(diff, ord='fro') / np.linalg.norm(C_orig, ord='fro')


# save output

def write_covariance_matrix(file_path, cov_matrix):
    """
    Writes the covariance matrix to file in the format:
      - First line: n (matrix dimension)
      - Then n*n lines: each matrix element (row by row).
    """
    n = cov_matrix.shape[0]
    with open(file_path, 'w') as file:
        # Write the dimension
        file.write(str(n) + "\n")
        
        # Write the matrix entries
        for i in range(n):
            for j in range(n):
                # You can adjust the formatting to your preference:
                # e.g., "{:.6e}" for scientific notation with 6 decimal places
                file.write(f"{cov_matrix[i, j]:.6e}\n")

## Diagonal shift tests
# further to decide the shift we examing the diagonal value and thier scale
def suggest_alpha(cov_matrix, fraction=1e-6):
    # Quick guess of scale based on median diagonal
    diag_vals = np.diag(cov_matrix)
    median_diag = np.median(diag_vals)
    if median_diag <= 0:
        # fallback: use absolute value or some default
        median_diag = np.mean(np.abs(diag_vals))
    return fraction * median_diag

# Diagonal shifting
def add_diagonal_shift(C, alpha=3.4529999999999996e-11):
    """
    Adds alpha * I to the matrix C. 
    This pushes all eigenvalues up by alpha.
    """
    n = C.shape[0]
    return C + alpha * np.eye(n)


In [6]:
file_path = 'covsys_000.txt'

In [7]:
# Use the function to read the covariance matrix
cov_matrix_list = read_covariance_matrix(file_path)

# Convert the list-of-lists to a NumPy array
cov_matrix = np.array(cov_matrix_list, dtype=float)
print("Shape of the covariance matrix:", cov_matrix.shape)

# Check if your matrix is positive definite
if is_positive_definite(cov_matrix):
    print("Matrix is already positive definite.")
else:
    print("Matrix is NOT positive definite.")


Shape of the covariance matrix: (5678, 5678)
Matrix is NOT positive definite.


In [18]:
## Checking the corresponding Hubble diagram is correct( by checking dimensions)
hd = pd.read_csv('output_hubble_diagram.txt',comment='#',sep='\s+')
print(hd.head(2))
print('\n\n Shape = ',np.shape(hd))

  VARNAMES:     CID  IDSURVEY     zCMB     zHEL        MU   MUERR  MUERR_VPEC  \
0       SN:  103327        50  0.01995  0.01995  34.89055  0.1317      0.0733   
1       SN:   94200        50  0.02209  0.02209  34.83914  0.1237      0.0663   

   MUERR_SYS    HOST_RA   HOST_DEC  
0    0.00646  55.591831 -32.440624  
1    0.00595  55.045319 -32.409492  


 Shape =  (5678, 11)


## Repairing the Matrix

### Eignevalue clipping

In [8]:
fixed_cov_matrix = clip_eigenvalues(cov_matrix, min_eigval=1e-10)
print("Is fixed_cov_matrix positive definite?", is_positive_definite(fixed_cov_matrix))


rel_change = relative_frobenius_change(cov_matrix, fixed_cov_matrix)
print("Relative Frobenius change:", rel_change)


Is fixed_cov_matrix positive definite? True
Relative Frobenius change: 2.300731520299082e-06


#### This method works and the change is << 1%, so it is safe to apply this correction

#### Diagonal shifting
#### We will not use this method but keep it for future reference

In [9]:
alpha_guess = suggest_alpha(cov_matrix, fraction=1e-6)
print("Proposed alpha =", alpha_guess)

# Example usage:
alpha = 1e-8
shifted_cov_matrix = add_diagonal_shift(cov_matrix, alpha=alpha)
print("Is shifted_cov_matrix positive definite?", is_positive_definite(shifted_cov_matrix))


Proposed alpha = 3.478e-11
Is shifted_cov_matrix positive definite? False


## Save output

In [10]:
output_file = "covsys_000_fixed.txt"
write_covariance_matrix(output_file, fixed_cov_matrix) # Saving the Eigenvalue clipped matrix
print(f"Saved fixed covariance to {output_file}")

Saved fixed covariance to covsys_000_fixed.txt
